In [1]:
from collections import OrderedDict
from types import SimpleNamespace

cfg_dict = {
    "repo_dir": "/bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/dinov3",
    "hub_name": "dinov3_vitb16",
    "pretrained_weights_path": "/bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/dinov3/checkpoints/dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth",
    "trained_checkpoint": "/bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/dinov3/results/da3_weight_1_0/eval/training_12499/teacher_checkpoint.pth",
}
cfg = SimpleNamespace(**cfg_dict)


import torch.hub
import torch

encoder = torch.hub.load(
    cfg.repo_dir, cfg.hub_name, source="local", weights=cfg.pretrained_weights_path
)

state = torch.load(cfg.trained_checkpoint, map_location="cpu")
teacher = state.get("teacher")
if teacher is None:
    raise KeyError("trained_checkpoint missing 'teacher' key")
prefix = "backbone."
new_state = OrderedDict(
    (k[len(prefix) :] if k.startswith(prefix) else k, v) for k, v in teacher.items()
)
missing, unexpected = encoder.load_state_dict(new_state, strict=False)



Downloading: "file:///bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/dinov3/checkpoints/dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth" to /home/nfs/surgicaldinov2/.cache/torch/hub/checkpoints/dinov3_vitb16_pretrain_lvd1689m-73cec8be.pth


100%|██████████| 327M/327M [00:03<00:00, 95.2MB/s] 


In [3]:
inputs = torch.randn(3, 3, 224, 224)

In [4]:
outputs = encoder.get_intermediate_layers(inputs, n = [5, 7, 9, 11])

In [7]:
outputs[0].shape

torch.Size([3, 196, 768])

In [1]:
import webdataset as wds

In [5]:
from data_scared import ScaredDepthDataModule

In [6]:
from dataclasses import dataclass
from omegaconf import DictConfig, OmegaConf

@dataclass
class MyConfig:
    name: str
    version: float
    is_active: bool = True  # 可以有默认值

In [7]:
# 模拟一个 DictConfig 对象
data = {
    "name": "project_alpha",
    "version": 1.5,
    "is_active": False
}
dict_config = OmegaConf.create(data)

# 确认类型
print(f"原始类型: {type(dict_config)}")
# 原始类型: <class 'omegaconf.dictconfig.DictConfig'>

原始类型: <class 'omegaconf.dictconfig.DictConfig'>


In [10]:
cfg_structured = OmegaConf.merge(OmegaConf.structured(MyConfig), dict_config)

# 4. 转换为 Dataclass 实例
app_config_instance = OmegaConf.to_object(cfg_structured)

In [11]:
app_config_instance

MyConfig(name='project_alpha', version=1.5, is_active=False)